In [1]:
import pandas as pd 
from pathlib import Path
import numpy as np 
import matplotlib.pyplot as plt 
import xarray as xr
import hydroeval as he 
import seaborn as sns 
from tqdm import tqdm 

In [2]:
## TO DO  
## - with subsampling, slightly increase nr 0s? for not occuring gauges 
## - if grid_format: return results of classification (p0, p1, y_hat)

In [3]:
## SETTINGS 

## which dataset to use 
## 1 --> most samples, less features 
## 2 --> less samples (about 20 less?), complete set of features 
use_set_1 = True

## omit coordinates as feature 
omit_coords = True

## set size of buffer search 
set_buffer_size = 6

## what similarity to use: diff, abs, ratio-1, ratio-2, double 
similarity_method = 'diff'

## grid training or cell training 
grid_format = True

## apply PCA 
do_pca = True 

Load dataset with features. On each row, a different simulation of observation sample can be found, in each column feature values or location metadata is stored

In [4]:
signature_dir = Path(r"C:\Users\mvand\Documents\Master EE\Year 4\Thesis\data\training_data_S1\signatures_nc_V1_output") 
 
if use_set_1:
    fn_signatures = signature_dir / "S1_merged_signatures_v2-cleanup-1.csv"
else:
    fn_signatures = signature_dir / "S1_merged_signatures_v2-cleanup-2.csv" 

df_signatures = pd.read_csv(fn_signatures, index_col = 0)

df_signatures

,Nm-all,Ns-all,N-gof-all,Lm-all,Ls-all,L-gof-all,Gu-all,Ga-all,Gev-gof-all,Gk-all,...,flv-seasonal_1,hf-f-all,hf-f-seasonal_1,hf-t-all,lf-f-all,lf-f-seasonal_1,lf-t-seasonal_1,pks-all,tag,dem_flag
ID,,,,,,,,,,,,,,,,,,,,,
6119010_11,102.035362,106.090645,0.0,4.221500,0.863208,0.0,54.306767,0.012089,0.0,0.925012,...,164.721008,0.590909,0.0,2.954545,0.363636,0.636364,4.481481,666.803894,6119010,0
6119010_12,100.917046,105.168777,0.0,4.209119,0.864435,0.0,53.603184,0.012195,0.0,0.920779,...,163.700439,0.590909,0.0,3.090909,0.363636,0.636364,4.518519,654.762268,6119010,0
6119010_13,0.544169,0.542255,0.0,-1.002645,0.867551,0.0,0.300217,2.365214,0.0,1.007072,...,266.693390,0.409091,0.0,1.727273,0.409091,0.636364,6.000000,3.131104,6119010,0
6119010_14,162.321777,138.910507,0.0,4.760090,0.835850,0.0,99.828016,0.009233,0.0,1.365474,...,317.679565,0.045455,0.0,0.136364,1.000000,0.545455,7.227273,723.082581,6119010,0
6119010_15,17.163115,16.641987,0.0,2.442999,0.888787,0.0,9.676133,0.077067,0.0,1.063609,...,248.398575,0.136364,0.0,0.363636,0.454545,0.500000,6.083333,98.092957,6119010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983350_96,522.018127,325.349060,0.0,6.080506,0.592225,0.0,375.648448,0.003942,0.0,2.574376,...,60.206692,0.000000,0.0,0.000000,0.111111,0.000000,0.000000,1890.435181,6983350,0
6983350_97,1.769354,1.644365,0.0,0.271318,0.743112,0.0,1.029579,0.779967,0.0,1.157798,...,73.878761,0.333333,0.0,1.000000,0.111111,0.555556,6.181818,10.525513,6983350,0
6983350_98,6.406629,5.884712,0.0,1.530997,0.797030,0.0,3.759185,0.217946,0.0,1.185247,...,92.737022,0.444444,0.0,1.000000,0.444444,0.444444,4.800000,33.151855,6983350,0


Slightly expand the searching area by labeling the up- and downstream neigbhours of the target pixel as 1

In [5]:
## expand searching area by max. 2 pixels 

def expand_labels(df, id_col, target_col, critical_variable, 
                  p_diff = 5., n_max = 3, pixel_size = 5000., option = 1,
                  x_coord = 'x', y_coord = 'y'):
    
    out_col = 'range_{}'.format(target_col)
    df[out_col] = 0.
    
    for bix in df[id_col].unique():
        
        ## select buffer and target value 
        _buffer = df[ df[id_col] == bix ] 
        
        ## discard gauge values 
        buffer_ix = [ix for ix in _buffer.index if not 'gauge' in ix]
        _sim = _buffer.loc[buffer_ix] 
        
        ## select target row 
        target = _sim[ _sim[target_col] == 1.]
        
        ## if target value found
        if len(target) > 0:
            
            ## search criterium 
            target_var = target[critical_variable].values[0] 
            
            ## look to adjacent pixels only 
            target_X_coord = target[x_coord].values[0]
            target_Y_coord = target[y_coord].values[0] 
            x_filter = (_sim[x_coord] >= target_X_coord - (pixel_size+1.) ) & (_sim[x_coord] <= target_X_coord + (pixel_size+1.))
            y_filter = (_sim[y_coord] >= target_Y_coord - (pixel_size+1.) ) & (_sim[y_coord] <= target_Y_coord + (pixel_size+1.))
            _sim = _sim[ x_filter & y_filter]

            ## calculate absolute percentual difference 
            d_var = (( ((_sim[critical_variable] - target_var)**2)**0.5 / target_var)*100 ).sort_values()
            
            ## OPTION 1 - select top three (including original target pixel) based on difference 
            if option == 1:
                expand_targets = d_var.head(n_max).index 

            ## OPTION 2 - also include a percentual limit (1% - 5%) 
            if option == 2:
                expand_targets = d_var[ d_var <= p_diff].index
                if len(expand_targets) > n_max:
                    expand_targets = expand_targets[:int(n_max-1)]

            ## relabel 
            if target.index[0] in expand_targets:
                df.loc[expand_targets, out_col] = 1 
            else:
                print('check ', bix)            

    return df, out_col

In [6]:
df_signatures, range_col = expand_labels(df_signatures, 'tag', 'target', 'Nm-all', option = 1)#, p_diff = 1.) 

Set the label categories - identify feature columns and target labels. Used for later processing of similarity and splitting X and y

In [7]:
## column descriptors 
columns = df_signatures.columns.values 

## target value to predict 
target_col = ['target'] 

## target for continued search 
target_buffer = ['in_buffer']

## should be omitted from dataset - but could be useful for easy selection 
## dem_flag only 1 for gauge values
non_feature_cols = ['n_buffer', 'tag', 'dem_flag', range_col, 'in_buffer'] 

## coord cols - could be ommitted from dataset, risk for overfitting
## and also takes away focus on selection based on timeseries
coord_cols = ['x', 'y', 'lat', 'lon']

## features that do not have to be transformed by subtraction 
## and cross-correlation properties, that are already a similarity property 
## as they are cross-correlation values of simulations and observations
non_similarity_cols = [col for col in columns if 'clag' in col]

feature_cols = [col for col in columns if (col not in non_feature_cols) & (not 'target' in col)]

## if coord_cols ommitted:
if omit_coords:
    feature_cols = [col for col in feature_cols if col not in coord_cols] 

## columns on which to perform similarity calculations:
calc_cols = [col for col in feature_cols if col not in non_similarity_cols]

Set the similarity metric used by the algorithm to find a best matching simulation

In [8]:
## calculate similarity per buffer 


def calc_similarity(df, buffer_col, calc_cols,
                   methods = ['diff']):
    
    df_out = df.copy() 
    
    buffer_idx = df[buffer_col].unique() 
    
    for ix in buffer_idx:
        df_buffer = df[ df[buffer_col] == ix] 
        
        cell_index = [row for row in df_buffer.index if not 'gauge' in row]
        gauge_index = [row for row in df_buffer.index if 'gauge' in row] 
        
        if len(gauge_index) > 0:
                
            if 'diff' in methods:            
                for col in calc_cols:
                    df_out.loc[cell_index, 'diff_{}'.format(col)] = df_out.loc[cell_index, col] - df_out.loc[gauge_index, col].values

            if 'abs' in methods:
                for col in calc_cols:
                     df_out.loc[cell_index, 'abs_{}'.format(col)] = ((df_out.loc[cell_index, col] - df_out.loc[gauge_index, col].values)**2)**0.5 

            if 'ratio-1' in methods:
                for col in calc_cols:
                    df_out.loc[cell_index, 'rat1_{}'.format(col)] = 1 - (( df_out.loc[cell_index, col] / (df_out.loc[gauge_index, col].values+1e-6)  )**2)**0.5

            if 'ratio-2' in methods:
                for col in calc_cols:
                    df_out.loc[cell_index, 'rat2_{}'.format(col)] =  ( df_out.loc[cell_index, col] - df_out.loc[gauge_index, col].values ) / (df_out.loc[gauge_index, col].values+1e-6)
            
            if 'double' in methods: 
                for col in calc_cols:
                    df_out.loc[cell_index, 'sim_{}'.format(col)] = df_out.loc[cell_index, col]
                    df_out.loc[cell_index, 'obs_{}'.format(col)] = df_out.loc[gauge_index, col].values[0]
        
        else:
            print('No gauge? :', ix)
            df_out = df_out.drop(index=cell_index)
                    
    ## drop calc_cols
    df_out = df_out.drop(columns=calc_cols)
    
    gauge_idx = [row for row in df_out.index if 'gauge' in row] 
    return df_out.drop(index=gauge_idx)

In [9]:
df_similarity = calc_similarity(df_signatures, 'tag', calc_cols, methods=[similarity_method])

df_similarity

,clag-0-all,clag-1-all,clag-0-seasonal_1,clag-1-seasonal_1,n_buffer,x,y,lat,lon,target,...,diff_fhv-seasonal_1,diff_flv-all,diff_flv-seasonal_1,diff_hf-f-all,diff_hf-f-seasonal_1,diff_hf-t-all,diff_lf-f-all,diff_lf-f-seasonal_1,diff_lf-t-seasonal_1,diff_pks-all
ID,,,,,,,,,,,,,,,,,,,,,
6119010_11,0.635464,0.544323,0.672078,0.537085,4.0,3422500.0,2352500.0,43.678226,-1.161527,0.0,...,8108.892578,-1091.800362,47.776888,0.454545,-0.045455,2.818182,0.318182,0.636364,4.481481,328.303894
6119010_12,0.635464,0.544067,0.671657,0.536465,4.0,3422500.0,2347500.0,43.633678,-1.152881,0.0,...,7921.990234,-1094.935036,46.756319,0.454545,-0.045455,2.954545,0.318182,0.636364,4.518519,316.262268
6119010_13,0.589989,0.522277,0.698611,0.582520,4.0,3422500.0,2342500.0,43.589130,-1.144252,0.0,...,-14805.981384,-713.167183,149.749269,0.272727,-0.045455,1.590909,0.363636,0.636364,6.000000,-335.368896
6119010_14,0.709440,0.570481,0.822756,0.639700,4.0,3422500.0,2337500.0,43.544575,-1.135641,0.0,...,11030.318359,-133.411201,200.735445,-0.090909,-0.045455,0.000000,0.954545,0.545455,7.227273,384.582581
6119010_15,0.612656,0.531497,0.700380,0.568576,4.0,3422500.0,2332500.0,43.500019,-1.127047,0.0,...,-11723.430176,-538.359810,131.454454,0.000000,-0.045455,0.227273,0.409091,0.500000,6.083333,-240.407043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983350_95,0.203398,0.227645,0.419789,0.451424,4.0,6502500.0,2872500.0,45.141037,38.478489,0.0,...,-14710.986877,-476.378696,48.027616,0.777778,0.111111,1.555556,0.000000,0.555556,6.000000,-990.893250
6983350_96,0.574453,0.580212,0.581299,0.579456,4.0,6502500.0,2867500.0,45.099716,38.455753,0.0,...,-2752.287109,-400.673678,29.618272,0.000000,0.000000,0.000000,-0.111111,0.000000,0.000000,897.935181
6983350_97,0.234536,0.260122,0.485846,0.515676,4.0,6502500.0,2862500.0,45.058388,38.433064,0.0,...,-14548.157715,-373.953403,43.290342,0.333333,0.000000,1.000000,-0.111111,0.555556,6.181818,-981.974487


Import required scikit-learn functions

In [10]:
# preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.decomposition import PCA

# algorithms 
from sklearn.linear_model import LogisticRegression 
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

# evaluation 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, balanced_accuracy_score 

Split a training & evaluation and test set. First investigate location of targets. Split datasets based on those distributions for equal division

In [11]:
## first split data based on being inside or outside the buffer zone (=4)
df_outside_buffer = df_similarity[ df_similarity['target'] == -1].copy() 
gauge_outside = df_outside_buffer['tag'].unique().astype(int)

fn_gauge = Path(r"C:\Users\mvand\Documents\Master EE\Year 4\Thesis\data\training_data_S1")  / "V1_grdc_efas_selection-cartesius-snapped-1.csv" 
meta_gauge = pd.read_csv(fn_gauge, index_col = 0)

## analyse large shifts 
df_meta_gauge = meta_gauge.loc[gauge_outside]
print('{} gauges / buffers fall outside the buffer zone of 4:'.format(len(gauge_outside)))
print(df_meta_gauge[['d_X_cell', 'd_Y_cell']].describe())
# print(df_meta_gauge[['d_X_cell', 'd_Y_cell']])

## copy data to new dataframe 
df_predict_buffer = df_similarity.copy() #.drop(index=df_outside_buffer.index) 
## set all outside buffer vals to 0 
df_predict_buffer.loc[df_outside_buffer.index, 'target'] = 0. 
## add target column for in-buffer prediction 
df_predict_buffer['in_buffer'] = 1 
df_predict_buffer.loc[ df_predict_buffer['tag'].isin(gauge_outside)  , 'in_buffer'] = 0. 

print('\nAnalyse size of buffer or searching range with number of hits:')
for buffer_size in np.sort(df_predict_buffer['n_buffer'].unique()):
    n_hits =df_predict_buffer[ (df_predict_buffer['n_buffer'] == buffer_size) & (df_predict_buffer['target']==1.) ]['target'].sum()
    print('In a search range of {:.0f} cells, {:.0f} matches are found'.format(buffer_size, n_hits))


## get buffers based on location of match 
buffer_0 = df_predict_buffer[ (df_predict_buffer['n_buffer'] == 0) & (df_predict_buffer['target']==1) ]['tag'].unique()
buffer_1 = df_predict_buffer[ (df_predict_buffer['n_buffer'] == 1) & (df_predict_buffer['target']==1) ]['tag'].unique()
buffer_2 = df_predict_buffer[ (df_predict_buffer['n_buffer'] == 2) & (df_predict_buffer['target']==1) ]['tag'].unique()
buffer_3 = df_predict_buffer[ (df_predict_buffer['n_buffer'] == 3) & (df_predict_buffer['target']==1) ]['tag'].unique()
buffer_4 = df_predict_buffer[ (df_predict_buffer['n_buffer'] == 4) & (df_predict_buffer['target']==1) ]['tag'].unique()

## split with same ratio 
train_val_0, test_0 = train_test_split(buffer_0, test_size=0.15, random_state=21)
train_val_1, test_1 = train_test_split(buffer_1, test_size=0.15, random_state=21)
train_val_2, test_2 = train_test_split(buffer_2, test_size=0.15, random_state=21)
train_val_3, test_3 = train_test_split(buffer_3, test_size=0.15, random_state=21)
train_val_4, test_4 = train_test_split(buffer_4, test_size=0.15, random_state=21)
train_val_out, test_out = train_test_split(gauge_outside, test_size=0.15, random_state=21)

## combine all sets 
train_val_sets = (train_val_0, train_val_1, train_val_2, train_val_3, train_val_4, train_val_out)
test_sets = (test_0, test_1, test_2, test_3, test_4, test_out)

id_train_val = np.concatenate( train_val_sets )
id_test = np.concatenate( test_sets )

## split data 
df_train_val = df_predict_buffer[ df_predict_buffer['tag'].isin(id_train_val)].copy()
df_test = df_predict_buffer[ df_predict_buffer['tag'].isin(id_test) ].copy()

## VARY SEARCHING AREA
## drop all rows with n_buffer values > set_buffer_size
## if match outside buffer, remaining set will be all zeros --> no buffer in current search area 
df_train_val = df_train_val[ df_train_val['n_buffer'] <= set_buffer_size ].copy()
df_test = df_test[ df_test['n_buffer'] <= set_buffer_size ].copy()

print('\nShow result of split:')
print('Train val set:')
print( df_train_val[df_train_val['target']==1].groupby(by='n_buffer')['target'].sum() )
print('\nTest set')
print( df_test[df_test['target']==1].groupby(by='n_buffer')['target'].sum() )

## list feature columns 
feature_columns = df_similarity.columns.values 

## remove non_feature cols 
if omit_coords:
    for col in non_feature_cols+target_col+coord_cols:
        col_ix = np.where(feature_columns==col)[0]
        feature_columns = np.delete(feature_columns, col_ix)

else:
    for col in non_feature_cols+target_col:
        col_ix = np.where(feature_columns==col)[0]
#         feature_columns = np.delete(feature_columns, col_ix)    

23 gauges / buffers fall outside the buffer zone of 4:
         d_X_cell    d_Y_cell
count   23.000000   23.000000
mean   -20.826087   -6.086957
std     55.257786   54.266106
min   -214.000000 -250.000000
25%    -11.500000   -1.000000
50%     -4.000000    6.000000
75%      2.000000   10.500000
max     13.000000   30.000000

Analyse size of buffer or searching range with number of hits:
In a search range of 0 cells, 244 matches are found
In a search range of 1 cells, 284 matches are found
In a search range of 2 cells, 32 matches are found
In a search range of 3 cells, 9 matches are found
In a search range of 4 cells, 3 matches are found

Show result of split:
Train val set:
n_buffer
0.0    207.0
1.0    241.0
2.0     27.0
3.0      7.0
4.0      2.0
Name: target, dtype: float64

Test set
n_buffer
0.0    37.0
1.0    43.0
2.0     5.0
3.0     2.0
4.0     1.0
Name: target, dtype: float64


In [12]:
# all_ix = np.concatenate((id_train_val, id_test))

# subset = meta_gauge.loc[all_ix, ['d_X_cell', 'd_Y_cell']]

# subset['buffer_shift'] = 0 
# x0 = 0 
# y0 = 0 

# for ix in subset.index:
#     row = subset.loc[ix]
    
#     x_shift = ((row['d_X_cell'])**2)**0.5 
#     y_shift = ((row['d_Y_cell'])**2)**0.5 
#     max_shift = np.max([x_shift, y_shift])
      
#     subset.loc[ix,'buffer_shift'] = max_shift
        
# analyse_distances = subset['buffer_shift'].value_counts()

# x_vals = analyse_distances.index
# y_vals = analyse_distances.values 


# save_dir = Path(r"C:\Users\mvand\Documents\Master EE\Year 4\Thesis\meetings\2021_06_15_EWC_sympo\media") 
# fn_1 = save_dir / 'shift_bins.png'
# fn_2 = save_dir / 'shift_bins_zoom.png'

# plt.figure(figsize=(12,4))
# plt.bar(x_vals, y_vals)
# plt.yscale('log')
# plt.grid()
# plt.xlabel('Shift nearest cell to matched cell', size=14);
# plt.ylabel('n observations (log-scaled)', size=14);
# plt.xticks(size=12);
# plt.yticks(size=12);
# plt.savefig(fn_1);

# plt.figure(figsize=(8,4))
# plt.bar(x_vals, y_vals)
# plt.xlim(-1, 5)
# plt.grid()
# plt.xlabel('Shift nearest cell to matched cell', size=14);
# plt.ylabel('n observations', size=14);
# plt.xticks(size=12);
# plt.yticks(size=12);
# plt.savefig(fn_2);

Load or calculate benchmark predictions

In [13]:
def benchmarks(df, id_col, target_col, obs_dir = None, sim_dir = None,
               methods = ['CC', 'NSE', 'RMSE', 'KGE'],
               fn_out = None): 
    
    df_out = pd.DataFrame() 
    
    df_out['ID'] = df.index.values 
    df_out[id_col] = df[id_col].values 
    df_out[target_col] = df[target_col].values 
    df_out = df_out.set_index('ID')
    
    buffer_idx = df_out[id_col].unique()
    
    for i in tqdm(range(len(buffer_idx))):
        buffer_id = buffer_idx[i]
        
        _out = df_out[ df_out[id_col] == buffer_id] 
                
        for method in methods:
            
            out_col = '{}_hat-{}'.format(target_col, method)
            
            if method in ['NSE', 'RMSE', 'KGE']:
                
                assert obs_dir.exists() , '[ERROR] obs_dir not specified or not found'
                assert sim_dir.exists(), '[ERROR] sim_dir not specified or not found'
                
                fn_obs = obs_dir / '{}_Q_Day.Cmd.txt'.format(buffer_id)
                fn_sim = sim_dir / 'buffer_{}_size-4.nc'.format(buffer_id) 
                
                ## load observations
                df_obs = pd.read_csv(fn_obs, skiprows=36, delimiter=';', encoding='cp850')
                df_obs['Q_obs'] = df_obs[' Value'] 
                df_obs['date'] = pd.to_datetime(df_obs['YYYY-MM-DD'], yearfirst=True,
                                               format='%Y-%m-%d')
                df_obs = df_obs.drop(columns=[' Value', 'YYYY-MM-DD', 'hh:mm'])
                df_obs = df_obs.set_index('date')
                df_obs.loc[ df_obs['Q_obs'] == -999., 'Q_obs'] = np.nan 
                df_obs = df_obs.loc[df_obs.index >= '1991']
                
                ## load simulations 
                ds_sim = xr.open_dataset(fn_sim)
                df_sim_list = ds_sim.to_dataframe().reset_index() 
                
                df_sim = pd.DataFrame()
                df_sim['date'] = pd.to_datetime( df_sim_list['time'].unique() )
                df_sim  = df_sim.set_index('date')
                
                for i, x_cell in enumerate( df_sim_list['x'].unique() ):
                    for j, y_cell in enumerate( df_sim_list['y'].unique() ):
                        
                        _df = df_sim_list[ (df_sim_list['x'] == x_cell) & (df_sim_list['y'] == y_cell) ]
                        
                        cell_id = '{}_{}{}'.format(buffer_id, int(i+1), int(j+1)) 
                        time = pd.to_datetime(_df['time'])
                        df_sim.loc[time, cell_id] = _df['dis24'].values 
                
                ## set simulations to observations extent 
                max_date = df_obs.tail(1).index.values[0]
                min_date = df_obs.head(1).index.values[0]
                df_sim = df_sim.loc[ (df_sim.index>=min_date) & (df_sim.index <= max_date)].copy()    
                
                ## mask and drop NaN values based on observations 
                gauge_mask = df_obs[df_obs['Q_obs'].isnull()>0].index 
                df_obs = df_obs.drop(index=gauge_mask)
                df_sim = df_sim.drop(index=gauge_mask)   
                
                ## calculate selected metrics in the buffer 
                for cell in df_sim.columns:
                    
                    if method == 'NSE':
                        try:
                            res = he.evaluator(he.nse, df_sim[cell].values, df_obs['Q_obs'].values )[0]
                        except:
                            print(buffer_id)
                            print(df_obs.head(2))
                            print(df_obs.tail(2))
                            print(df_sim.head(2).index)
                            print(df_sim.tail(2).index)
                            
                        
                    if method == 'RMSE':
                        res = he.evaluator(he.rmse, df_sim[cell].values, df_obs['Q_obs'].values )[0]  
                        
                    if method == 'KGE':
                        res, r, alpha, beta = he.evaluator(he.kge, df_sim[cell].values, df_obs['Q_obs'].values ) 

                    df_out.loc[cell, method] = res 
                
                if method == 'NSE':
                    y_hat_ix = df_out.loc[_out.index, method].idxmax()
                if method == 'RMSE':
                    y_hat_ix = df_out.loc[_out.index, method].idxmin()
                if method == 'KGE':
                    y_hat_ix = df_out.loc[_out.index, method].idxmax()
                        
                
                df_out.loc[_out.index, out_col] = 0 
                df_out.loc[y_hat_ix, out_col] = 1

            if method == 'CC': 
                df_out.loc[ _out.index, out_col ] = 0 
                center_ix = df[ (df['n_buffer'] == 0) & (df[id_col]==buffer_id)].index
                df_out.loc[center_ix, out_col] = 1 
            
    df_out = df_out.dropna(axis=0)
                
    if fn_out == None:
        return df_out
    else:
        df_out.to_csv(fn_out)
        return fn_out 

In [14]:
## load or calculate benchmarks 
calc_bench = False
fn_bench = signature_dir / "S1_benchmarks.csv" 

if calc_bench:

    base_dir = Path(r"C:\Users\mvand\Documents\Master EE\Year 4\Thesis\data\training_data_S1") 

    obs_dir = base_dir / "V1" 
    sim_dir = base_dir / "efas_output_nc"

    df_benchmarks = benchmarks(df_similarity, 'tag', target_col[0], 
                               obs_dir, sim_dir) #, fn_out = fn_bench)

else:
    df_benchmarks = pd.read_csv(fn_bench, index_col=0)

df_benchmarks

df_benchmarks['range_target'] = df_similarity['range_target']
df_benchmarks.loc[ df_benchmarks['range_target'].isnull() ] = 0 

benchmarks_train_val = df_benchmarks[df_benchmarks['tag'].isin(id_train_val)]
benchmarks_test = df_benchmarks[df_benchmarks['tag'].isin(id_test)]

If necessary, data can be transformed into a grid for training

In [15]:
def feature_grid(df, feature_cols, target_col, range_col=None, n_elements=9):
    
    ## create  multidimensional array with NaN value: -999. 
    feature_grid = np.ones((len(feature_cols), n_elements, n_elements)) * -999. 
    target_grid = np.zeros((n_elements, n_elements))
    
    range_grid = None 
    if range_col != None:
        range_grid = np.zeros((n_elements, n_elements))
        
    ## fill array 
    for ix in df.index:
        grid_x, grid_y = int(ix.split('_')[-1][0]), int(ix.split('_')[-1][1]) 
                
        feature_grid[:, int(grid_y-1), int(grid_x-1) ] = df.loc[ix, feature_cols].values 
        target_grid[int(grid_y-1),int(grid_x-1)] = df.loc[ix, target_col] 
        
        if range_col!= None:
            range_grid[int(grid_y-1),int(grid_x-1)] = df.loc[ix, range_col]
                     
    return feature_grid, target_grid, range_grid

def reshape_to_grid(df, feature_cols, target_col, range_col=None, id_col='tag', buffer_size=4):
    
    n_elements = int(1 + (2*buffer_size))
    
    ## create emtpy output grids 
    grid_features = np.zeros((df[id_col].nunique(), len(feature_cols), n_elements, n_elements )) 
    grid_targets = np.zeros((df[id_col].nunique(), n_elements, n_elements))
    
    grid_range = None
    if range_col != None:
        grid_range = np.zeros((df[id_col].nunique(), n_elements, n_elements))
    
    ## reshape each buffer 
    for i, idx in tqdm(enumerate(df[id_col].unique())):
                
        df_buffer = df[ df[id_col] == idx ] 
        
        grid_buffer_features, grid_buffer_target, grid_buffer_range = feature_grid(df_buffer, feature_cols, 
                                                                                   target_col, range_col, n_elements) 
            
        grid_features[i] = grid_buffer_features
        grid_targets[i] = grid_buffer_target
        
        if range_col != None:
            grid_range[i] = grid_buffer_range
    
    if range_col == None:
        return grid_features, grid_targets 
    
    return grid_features, grid_targets, grid_range

In [16]:
if grid_format:
    
    ds_train_val = xr.Dataset()
    ds_test = xr.Dataset()
    
    ## retransform datarame to grids 
    n_elements = min(set_buffer_size, 4)
    
    ## CASE 1 
    ## for determining target in buffer 
    X1_train_val, y1_train_val= reshape_to_grid(df_train_val, feature_columns,
                                                target_buffer[0], buffer_size=n_elements) 
    X1_test, y1_test = reshape_to_grid(df_test, feature_columns, 
                                       target_buffer[0], buffer_size=n_elements) 
    
    ## flatten X1 
    n_samples, n_features, n_rows, n_cols = X1_train_val.shape 
    X1_train_val_flatten = np.reshape(X1_train_val, (n_samples, int(n_features*n_rows*n_cols) ) )
    X1_test_flatten = np.reshape(X1_test, (len(X1_test), int(n_features*n_rows*n_cols))  )
    
    
    ## SAVE AS XARRAY ?
#     coords, dummy_target = reshape_to_grid(df_train_val, ['x', 'y',],
#                                       target_buffer[0], buffer_size = n_elements)
#     print(coords)
    
    
    ## simplify y1 values to single vector 
    y1_train_val_vector = np.zeros(n_samples)
    for i in range(n_samples):
        y1_train_val_vector[i] = np.max(y1_train_val[i])
    
    y1_test_vector = np.zeros(len(X1_test))
    for i in range(len(y1_test_vector)):
        y1_test_vector[i] = np.max(y1_test[i])
    
    
    ## CASE 2 
    ## for determining location of cell 
    ## for determining target in buffer 
    X2_train_val, y2_train_val, y2_train_val_range = reshape_to_grid(df_train_val, feature_columns,
                                                                     target_col[0], range_col = range_col,
                                                                     buffer_size=n_elements) 
    
    X2_test, y2_test, y2_test_range = reshape_to_grid(df_test, feature_columns, 
                                                      target_col[0], range_col=range_col, 
                                                      buffer_size=n_elements) 
    
    ## flatten X2 
    n_samples, n_features, n_rows, n_cols = X2_train_val.shape 
    X2_train_val_flatten = np.reshape(X1_train_val, (n_samples, int(n_features*n_rows*n_cols) ) )
    y2_train_val_flatten = np.reshape(y2_train_val, (n_samples, int(n_rows*n_cols) ) )
    y2_train_val_range_flatten = np.reshape(y2_train_val_range, (n_samples, int(n_rows*n_cols) ) )
    
    n_test_samples = len(X2_test)
    X2_test_flatten = np.reshape(X2_test, (n_test_samples, int(n_features*n_rows*n_cols))  )  
    y2_test_flatten = np.reshape(y2_test, (n_test_samples, int(n_rows*n_cols) ) )
    y2_test_range_flatten = np.reshape(y2_test_range, (n_test_samples, int(n_rows*n_cols) ) )   
    
    
    ## SAVE AS XARRAY ?

503it [00:29, 16.78it/s]
92it [00:05, 16.16it/s]
503it [00:30, 16.65it/s]
92it [00:05, 16.49it/s]


In [17]:
## sum y_target vals to see distribution 
# x_ticks = np.arange(0,9) + 0.5 
# labels = np.arange(-4, 5, 1)

# train_sum = y2_train_val.sum(axis=0)
# test_sum = y2_test.sum(axis=0)
# total_sum = train_sum + test_sum

# plt.figure(figsize=(12,8))
# sns.heatmap( total_sum , cmap =  'Blues', annot=True, fmt='g', 
#            cbar = False, annot_kws={"size":16})

# plt.xticks(x_ticks, ['{}'.format(v) for v in labels], size = 14);
# plt.yticks(x_ticks, ['{}'.format(v) for v in labels][::-1], rotation=0, size = 14);
# plt.xlabel('Shift from center coordinate', size = 18);
# plt.ylabel('Shift from center coordinate', size = 18);

# save_dir = Path(r"C:\Users\mvand\Documents\Master EE\Year 4\Thesis\meetings\2021_06_15_EWC_sympo\media") 
# fn = save_dir / 'grid_shift_view.png'
# plt.savefig(fn)

Now, with k-fold cross validation find optimal algorithm settings

In [80]:
def benchmark_performances(y, y_hat, k, model_name, grid_format = False,
                          id_col='tag', target_col='range_target'):
    
    ## customize classifcation evaluation
    TP = 0 
    TN = 0
    FP = 0
    FN = 0 
    n_targets = 0 
    
    if grid_format:
        
        n_samples, n_classes = y.shape 
        n_cells = n_samples * n_classes 
        n_samples, n_classes = y.shape       
        
        ## reshape to count TP, TN, FP, FN 
        for sample in range(len(y)):
            
            row_y = y[sample]
            row_y_hat = y_hat[sample]
            
            ix_y = np.where(row_y > 0)[0]
            ix_y_hat = np.where(row_y_hat > 0)[0]
            
            ## positve target label exists 
            if len(ix_y) > 0:
                n_targets += 1 
            
                ## if a positive target label exists, and 
                ## prediction contains a positive label, 
                ## prediction is either true or false 
                if len(ix_y_hat) > 0:
                    
                    ## if prediction true, TP = 1 and TN are remaining cells 
                    if ix_y_hat[0] in ix_y:
                        TP += 1 
                        TN += (n_classes-1)
                    
                    ## if prediction is false, TP = 1 and TN are remaining cells 
                    if ix_y_hat[0] not in ix_y:
                        FP += 1                    # or sum(row_y) ?, so 1-3 false positives)
                        TN += (n_classes-1)        # or n_classes - sum(row_y)
                
                ## if a positive target label exists, but
                ## prediction is all zeros, false negatives are counted 
                else: 
                    FN += 1 
                    TN += (n_classes-1)
            
            ## no positive labels 
            if len(ix_y) == 0:
                ## and prediction correctly all zero as well 
                if len(ix_y_hat) == 0:
                    TN += n_classes 
                    
                ## else one label is incorrectly labelled positive 
                ## rest is true negative 
                else:
                    FP += 1
                    TN += (n_classes-1)
                    
    else:
        
        ## reshape to count TP, TN, FP, FN 
        y['hat'] = y_hat
        n_cells = len(y) 
        
        for ix in y[id_col].unique():
            _buffer = y[ y[id_col] == ix ]
            
            n_classes = len(_buffer)
            
            _y = _buffer[target_col]
            _y_hat = _buffer['hat']
            
            ## if positive target
            if _y.sum() > 0:
                n_targets += 1 
                _y_ix = _y[ _y>0].index.values 

                ## if positive prediction 
                if _y_hat.sum() > 0:
                    _y_hat_ix = _y_hat.idxmax() 
                    
                    ## if correct prediction 
                    if _y_hat_ix in _y_ix:
                        TP += 1 
                        TN += (n_classes-1)
                    ## incorrect prediction
                    else:
                        FP += 1
                        TN += (n_classes-1)
                        
                ## if no positive prediction
                else:
                    FN += 1 
                    TN += (n_classes-1)
                    
            ## if no positive targets 
            else:
                ## if no positive predictions 
                if _y_hat.sum() == 0:
                    TN += n_classes
                ## if positive predictions 
                else:
                    FP += 1
                    TN += (n_classes-1)
    
    ## calculate metrics 
    acc = (TP+TN) / (TP+TN+FP+FN)
    b_acc = 0.5 * ( (TP/(TP+FN)) + (TN/(TN+FP))  )
    prec = TP / (TP+FP)
    rec = TP / (TP+FN) 
    f1 = (prec*rec) / (prec+rec+1e-6) 
    hit_rate = TP / n_targets 

    return_df = pd.DataFrame({
                            'k': [k],
                            'model': [model_name],
                            'accuracy': [acc],
                            'balanced_acc': [b_acc],
                            'precision': [prec],
                            'recall': [rec],
                            'f1': [f1],
                            'hit_rate': [hit_rate],
                            'n': [n_targets],
                            'N': [n_cells],
                            'TP': [TP],
                            'TN': [TN],
                            'FP': [FP],
                            'FN': [FN]
                              })     
    return return_df


def buffer_classifier(df_val, y_prob, p0_col, p1_col,
                     prediction_col, id_col, prob_threshold):
        
    df_val[p0_col] = y_prob[:,0]
    df_val[p1_col] = y_prob[:,1] 
    df_val[prediction_col] = 0.
    
    for ix in df_val[id_col].unique():
        max_ix = df_val[ (df_val[id_col]==ix) ][p1_col].idxmax()
        
        if df_val.loc[max_ix, p1_col] >= prob_threshold:
            df_val.loc[max_ix, prediction_col] = 1.

    return df_val

def k_foldCV(X, y = [],  grid_format = False, K = 5, 
             id_col = None, feature_cols = [], target_col = None,
             range_col = None, y_range=[],
             n_subsample = 1, do_norm = True, do_scale = True, 
             do_PCA = True, n_pca = 0.9, whiten_pca = False,
             methods = ['LR-2'], prob_threshold = 0.5, 
             return_classification = False, gauge_id_list = []):
             
#              benchmarks = ['CC', 'NSE', 'RMSE', 'KGE'],
#              df_benchmarks = None):

    df_performance = pd.DataFrame()
    
    ## for returning classification 
    if grid_format:
        ds_collect_val = xr.Dataset()
        n_cells = y.shape[1] 
        ## assume square grid 
        n_buffer = int(n_cells**0.5)
    else:
        df_collect_val = pd.DataFrame()
        
    
    if grid_format:
        assert len(y) > 0, 'target column not specified'
        sort_ix = list(range(len(X)))
                
        
    if not grid_format:
        assert id_col != None, 'buffer id col not specified'
        assert target_col != None, 'buffer id col not specified'
        assert len(feature_cols) > 0, 'feature columns not specified'
        
        sort_ix = X[id_col].unique()
    
    
    ## shuffle and split ids 
    np.random.seed(26)
    np.random.shuffle(sort_ix)
    
    ## split in K folds 
    k_split_idx = np.array_split(sort_ix, K)
    
    for k in tqdm(range(K)):
        
        ## split samples 
        id_val = k_split_idx[k]
        id_train = np.setdiff1d(sort_ix, id_val)
        
        if grid_format:
            X_train = X[id_train]
            y_train = y[id_train]
            
            X_val = X[id_val]
            y_val = y[id_val]
            
            if len(y_range) > 0:
                y_val_range = y_range[id_val]
        
        if not grid_format:
            
            set_train = X[ X[id_col].isin(id_train) ]
            set_val = X[ X[id_col].isin(id_val)]
            
            ## subsample training set only 
            set_train_1 = set_train[ set_train[target_col] == 1 ]
            
            set_train_0_ix = []
            for i, idx in enumerate(id_train):
                _df = set_train[ (set_train[target_col] != 1) & (set_train[id_col] == idx) ] 
                subsamples = _df.sample(n=n_subsample).index 
                for sample in subsamples:
                    set_train_0_ix.append(sample)
            set_train_0 = set_train.loc[set_train_0_ix]
            
            ## sample in_buffer = 0 samples 
            set_not_in_buffer = set_train[set_train['in_buffer']!=1]
            n_sample = set_not_in_buffer[id_col].nunique()
            print(n_sample)

            ## add subsampled sets             
            sample_train = set_train_1.append(set_train_0)
            sample_train = sample_train.append( set_not_in_buffer.sample(n=n_sample) )
            
            ## split X and y 
            X_train = sample_train[feature_cols]
            y_train = sample_train[target_col]

            X_val = set_val[feature_cols]
            y_val = set_val[target_col]
            df_y_val = set_val.copy() #[[id_col, target_col]].copy()#.to_frame()

            if range_col is not None:
                y_val_range = set_val[[range_col, id_col]].copy()
        
        ## preprocess training data 
        if do_norm:
            min_samples = len(X_train)
            nm = QuantileTransformer(output_distribution='normal',
                                    n_quantiles = int(min(1000, min_samples)))
            X_train = nm.fit_transform(X_train)
        else:
            nm = None 
        
        if do_scale:
            sc = MinMaxScaler([0,1])
            X_train = sc.fit_transform(X_train)
        else:
            sc = None
        
        if do_PCA:
            pca = PCA(n_components = n_pca, whiten = whiten_pca)
            pca.fit(X_train)
            X_train = pca.transform(X_train)
        else:
            pca = None 
            
        ## prepare trainin data 
        if do_norm:
            X_val = nm.transform(X_val)
        if do_scale:
            X_val = sc.transform(X_val)
        if do_PCA:
            X_val = pca.transform(X_val)        
        
        ##  test models 
        for method in methods: 
            
            prediction_col = '{}_{}'.format(method, target_col)
            p0_col = '{}_p0'.format(method)
            p1_col = '{}_p1'.format(method)
                        
            if 'LR' in method:
                
                ## train model 
                lr = LogisticRegression(max_iter=1000)
                
                if grid_format:
                    lr = OneVsRestClassifier( lr )
                
                lr.fit(X_train, y_train)
                
                ## evaluate                                 
                if 'LR-1' in method:
                    y_val_hat = lr.predict(X_val)
                        
                if 'LR-2' in method:
                    y_val_hat_prob = lr.predict_proba(X_val)
                    
                    if grid_format:
                        y_val_hat = np.where( y_val_hat_prob >= prob_threshold, 1, 0)
                    else:
                        
                        df_y_val = buffer_classifier(df_y_val, y_val_hat_prob,
                                                    p0_col, p1_col, prediction_col,
                                                    id_col, prob_threshold)
                        
                        y_val_hat = df_y_val[prediction_col]  

                        
                
            if 'RF' in method:
                
                ## train model 
                rfc = RandomForestClassifier()                
                rfc.fit(X_train, y_train)
                
                if 'RF-1' in method:
                    y_val_hat = rfc.predict(X_val)
                     
                if 'RF-2' in method:
                    _y_val_hat_prob = rfc.predict_proba(X_val)
                    
                    if grid_format:
                        y_val_hat = np.zeros(y_val.shape)
                        y_val_hat_prob = np.zeros(y_val.shape)

                        for sample in range(len(_y_val_hat_prob)):
                            if _y_val_hat_prob[sample].shape[1] >1:
                                p1 = _y_val_hat_prob[sample][:,1]
                                _y_val_hat = np.zeros(len(p1)) 
                                
                                ix_max = p1.argmax() 
                                
                                if p1[p1.argmax()] >= prob_threshold:
                                    _y_val_hat[ix_max] = 1. 

                                y_val_hat[:,sample] = _y_val_hat 
                                y_val_hat_prob[:,sample] += p1 
                    
                    else:
                        y_val_hat_prob = _y_val_hat_prob 
                        df_y_val = buffer_classifier(df_y_val, y_val_hat_prob,
                                                    p0_col, p1_col, prediction_col,
                                                    id_col, prob_threshold)
                        
                        y_val_hat = df_y_val[prediction_col]
                    
            if 'SVM' in method:
                
                svc = SVC(probability=True)
                
                if grid_format:
                    svc = OneVsRestClassifier(svc) 
                
                svc.fit(X_train, y_train)
                
                if 'SVM-1' in method:
                    y_val_hat = svc.predict(X_val)
                    
                if 'SVM-2' in method:
                    y_val_hat_prob = svc.predict_proba(X_val)
                    
                    if grid_format:
                        y_val_hat = np.where( y_val_hat_prob >= prob_threshold, 1, 0)
                        
                    else:   
                        
                        df_y_val = buffer_classifier(df_y_val, y_val_hat_prob,
                                                    p0_col, p1_col, prediction_col,
                                                    id_col, prob_threshold)
                        
                        y_val_hat = df_y_val[prediction_col]                

            if 'k-nn' in method:
                
                if not do_PCA:
                    
                    if not do_norm:
                        min_samples = len(X_train)
                        nm = QuantileTransformer(output_distribution='normal',
                                                n_quantiles = int(min(1000, min_samples)))
                        X_train = nm.fit_transform(X_train)
                        X_val = nm.transform(X_train)
                    
                    if not do_scale:
                        sc = MinMaxScaler([0,1])
                        X_train = sc.fit_transorm(X_train)
                        X_val = sc.transform(X_val)
                    
                    pca = PCA(n_components = n_pca)
                    pca.fit(X_train)
                    X_train = pca.transform(X_train)
                    X_val = pca.transform(X_val)
                
                knn = KNeighborsClassifier() 
                                
                knn.fit(X_train, y_train) 
                
                if 'k-nn-1' in method:
                    y_val_hat = knn.predict(X_val)
                    
                if 'k-nn-2' in method:
                    _y_val_hat_prob = knn.predict_proba(X_val)
                    
                    if grid_format:
                        y_val_hat = np.zeros(y_val.shape)
                        y_val_hat_prob = np.zeros(y_val.shape)

                        for sample in range(len(_y_val_hat_prob)):
                            if _y_val_hat_prob[sample].shape[1] >1:
                                p1 = _y_val_hat_prob[sample][:,1]
                                _y_val_hat = np.zeros(len(p1)) 
                                
                                ix_max = p1.argmax() 
                                
                                if p1[p1.argmax()] >= prob_threshold:
                                    _y_val_hat[ix_max] = 1. 

                                y_val_hat[:,sample] = _y_val_hat
                                y_val_hat_prob[:,sample] += p1 
                    
                    else: 
                        y_val_hat_prob = _y_val_hat_prob 
                        df_y_val = buffer_classifier(df_y_val, y_val_hat_prob,
                                                    p0_col, p1_col, prediction_col,
                                                    id_col, prob_threshold)
                        
                        y_val_hat = df_y_val[prediction_col]
                    
            
                               
            ## calculate perforamnce 
            df_performance = df_performance.append( benchmark_performances(y_val_range, y_val_hat,
                                                                          k, method, grid_format=grid_format) )
            if not grid_format:
                try:
                    df_collect_val.loc[df_y_val.index, df_y_val.columns] = df_y_val 
                except:
                    df_collect_val = df_collect_val.append(df_y_val)
            
            else:
                 
                ## reshape data back to grid 
                n_samples = len(id_val)
                y_val_gridded = y_val.reshape((n_samples, n_buffer, n_buffer)) 
                y_val_hat_gridded = y_val_hat.reshape((n_samples, n_buffer, n_buffer))
                
                if len(y_range) > 0:
                    y_val_range_gridded = y_val_range.reshape((n_samples, n_buffer, n_buffer))
                
                try:
                    print(method)
                    y_hat_p1_gridded = y_val_hat_prob.reshape((n_samples, n_buffer, n_buffer))
                except:
                    y_hat_p1_gridded = [] 
                                
                for i, idv in enumerate(id_val):
        
                    ## layer name 
                    ## with index to retract original gauge id value 
                    if len(gauge_id_list) == 0:
                        layer_y_hat = f'{idv}_{method}_y_hat'
                        layer_y = f'{idv}_{method}_y' 
                        layer_y_range = f'{idv}_{method}_y_range'
                        layer_p1_prob = f'{idv}_{method}_p1'
                        
                    else:
                        gauge_id = gauge_id_list[idv]
                        layer_y_hat = f'{gauge_id}_{method}_y_hat'
                        layer_y = f'{gauge_id}_{method}_y'
                        layer_y_range = f'{gauge_id}_{method}_y_range'
                        layer_p1_prob = f'{gauge_id}_{method}_p1'
                    
                    ## reshape data to grid 
                    ds_collect_val[layer_y_hat] = (("x", "y"), y_val_hat_gridded[i] )
                    ds_collect_val[layer_y] = (("x", "y"), y_val_gridded[i] )
                    
                    if len(y_range) > 0:
                        ds_collect_val[layer_y_range] = (("x", "y"), y_val_range_gridded[i] )
                    
                    if len(y_hat_p1_gridded) > 0:
                        ds_collect_val[layer_p1_prob] = (("x", "y"), y_hat_p1_gridded[i])
                    

    if return_classification:
        if grid_format:
            return df_performance, ds_collect_val    
        else:
            return df_performance, df_collect_val                 
    else:
        return df_performance 


In [19]:
## K-fold settings 

n_folds = 1 
algorithms = ['LR-1', 'LR-2', 'RF-1', 'RF-2', 'SVM-1', 'SVM-2', 'k-nn-1', 'k-nn-2'] 

stat_cols = ['accuracy', 'balanced_acc', 'precision', 'recall', 'f1',  
             'TP', 'TN', 'FP', 'FN', 'hit_rate', 'n', 'N']

std_cols = ['accuracy', 'balanced_acc', 'precision', 'recall', 'f1',  
             'TP', 'TN', 'FP', 'FN', 'hit_rate']

test_dir = Path(r"C:\Users\mvand\Documents\Master EE\Year 4\Thesis\data\test_output\PCA_analysis")


In [ ]:
### no warnings: RF and k-nn, 
### LR1,2 give non-fatal warning, returns [n_samples, n_pixels/classes] for both  ?
### SVM1,2 give non-fatal warning, returns [n_samples, n_pixels/classes] for both ?
### RF-2 returns a list with length 81 (n_pixels/classes), and each item in list has lenght 101 (n_samples), with
### 1 or 2 columns 
### knn-2 returns a list with length 81 (n_pixels/classes), and each item in list has lenght 101 (n_samples), with
### 1 or 2 columns 

performance, ds_val = k_foldCV(X2_train_val_flatten,  y2_train_val_flatten, grid_format=True,
                        y_range = y2_train_val_range_flatten, methods=['k-nn-2'], 
                      gauge_id_list = id_train_val, return_classification=True) # methods = algorithms) 

  0%|                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

k-nn-2


 20%|████████████████████████████▏                                                                                                                | 1/5 [00:14<00:57, 14.36s/it]

k-nn-2


 40%|████████████████████████████████████████████████████████▍                                                                                    | 2/5 [00:41<01:06, 22.14s/it]

k-nn-2


In [73]:
performance.groupby(by='model')[stat_cols].mean()

,accuracy,balanced_acc,precision,recall,f1,TP,TN,FP,FN,hit_rate,n,N
model,,,,,,,,,,,,
k-nn-2,0.988268,0.506169,0.633333,0.012463,0.012189,1.2,8051.8,1.0,94.6,0.012355,96.8,8148.6


In [28]:
# _df, df_class_noPCA = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0],
#                                    range_col = 'range_target',feature_cols = feature_columns, 
#                                    methods = algorithms, do_PCA = True, return_classification=True)

# _df.groupby(by='model')[stat_cols].mean()

In [ ]:
## PCA sensitivity analysis - for raster pixel 

df_grid_no_pca = pd.DataFrame()
df_grid_pca60 = pd.DataFrame()
df_grid_pca70 = pd.DataFrame()
df_grid_pca80 = pd.DataFrame()
df_grid_pca90 = pd.DataFrame()
df_grid_pca95 = pd.DataFrame()
df_grid_pca99 = pd.DataFrame() 

for kn in range(n_folds):
    print('Iteration {}'.format(int(kn+1)))
    ## no pca 
#     _df = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], range_col = 'range_target',
#                   feature_cols = feature_columns, methods = algorithms, do_PCA = False)
#     df_grid_no_pca =df_grid_no_pca.append(_df)

#     ## PCA 60% 
#     _df = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], range_col = 'range_target',
#                   feature_cols = feature_columns, methods = algorithms, do_PCA = True, n_pca = 0.6 )
#     df_grid_pca60 = df_grid_pca60.append(_df)    

#     ## PCA 70% 
#     _df = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], range_col = 'range_target',
#                   feature_cols = feature_columns, methods = algorithms, do_PCA = True, n_pca = 0.7 )
#     df_gridle_pca70 = df_grid_pca70.append(_df) 

#     ## PCA 80% 
#     _df = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], range_col = 'range_target',
#                   feature_cols = feature_columns, methods = algorithms, do_PCA = True, n_pca = 0.8 )
#     df_grid_pca80 = df_grid_pca80.append(_df) 

#     ## PCA 90% 
#     _df = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], range_col = 'range_target',
#                   feature_cols = feature_columns, methods = algorithms, do_PCA = True, n_pca = 0.9 )
#     df_grid_pca90 = df_grid_pca90.append(_df) 

#     ## PCA 95% 
#     _df = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], range_col = 'range_target',
#                   feature_cols = feature_columns, methods = algorithms, do_PCA = True, n_pca = 0.95 )
#     df_grid_pca95 = df_grid_pca95.append(_df) 

#     ## PCA 99% 
#     _df = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], range_col = 'range_target',
#                   feature_cols = feature_columns, methods = algorithms, do_PCA = True, n_pca = 0.99 )
#     df_grid_pca99 = df_grid_pca99.append(_df) 


# list_df = [df_single_no_pca, df_single_pca60, df_single_pca70, df_single_pca80, df_single_pca90, df_single_pca95, df_single_pca99 ]

# list_name = ['alg_set-2_grid_noPCA.csv',
#             'alg_set-2_grid_60PCA.csv',
#             'alg_set-2_grid_70PCA.csv',
#             'alg_set-2_grid_80PCA.csv',
#             'alg_set-2_grid_90PCA.csv',
#             'alg_set-2_grid_95PCA.csv',
#             'alg_set-2_grid_99PCA.csv',]

# for i, out_df in enumerate(list_df):
    
#     fn = test_dir / list_name[i]
#     print(fn)
#     out_df.to_csv(fn)

In [ ]:
## PCA sensitivity analysis - for single pixel 

df_single_no_pca = pd.DataFrame()
df_single_pca60 = pd.DataFrame()
df_single_pca70 = pd.DataFrame()
df_single_pca80 = pd.DataFrame()
df_single_pca90 = pd.DataFrame()
df_single_pca95 = pd.DataFrame()
df_single_pca99 = pd.DataFrame() 

for kn in range(n_folds):
    print('Iteration {}'.format(int(kn+1)))
    ## no pca 
    _df, df_class_noPCA = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0],
                                   range_col = 'range_target',feature_cols = feature_columns, 
                                   methods = algorithms, do_PCA = False, return_classification=True)
#     df_single_no_pca = df_single_no_pca.append(_df)

#     ## PCA 60% 
#     _df, df_class_PCA60 = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], 
#                                    range_col = 'range_target', feature_cols = feature_columns, 
#                                    methods = algorithms, do_PCA = True, n_pca = 0.6, return_classification=True )
    
#     df_single_pca60 = df_single_pca60.append(_df)    

#     ## PCA 70% 
#     _df, df_class_PCA70 = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], 
#                                    range_col = 'range_target', feature_cols = feature_columns, 
#                                    methods = algorithms, do_PCA = True, n_pca = 0.7, return_classification=True)
#     df_single_pca70 = df_single_pca70.append(_df) 

#     ## PCA 80% 
#     _df, df_class_PCA80 = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0],
#                                    range_col = 'range_target', feature_cols = feature_columns,
#                                    methods = algorithms, do_PCA = True, n_pca = 0.8, return_classification=True )
#     df_single_pca80 = df_single_pca80.append(_df) 

#     ## PCA 90% 
#     _df, df_class_PCA90 = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], 
#                                    range_col = 'range_target', feature_cols = feature_columns, 
#                                    methods = algorithms, do_PCA = True, n_pca = 0.9, return_classification=True )
#     df_single_pca90 = df_single_pca90.append(_df) 

#     ## PCA 95% 
#     _df, df_class_PCA95 = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], 
#                                    range_col = 'range_target', feature_cols = feature_columns, 
#                                    methods = algorithms, do_PCA = True, n_pca = 0.95, return_classification=True )
#     df_single_pca95 = df_single_pca95.append(_df) 

#     ## PCA 99% 
#     _df, df_class_PCA99 = k_foldCV( df_train_val, id_col='tag', target_col = target_col[0], 
#                                    range_col = 'range_target', feature_cols = feature_columns,
#                                    methods = algorithms, do_PCA = True, n_pca = 0.99, return_classification=True )
#     df_single_pca99 = df_single_pca99.append(_df) 

In [ ]:
## SAVE RESULTS 
# list_df = [df_single_no_pca, df_single_pca60, df_single_pca70, df_single_pca80, df_single_pca90, df_single_pca95, df_single_pca99 ]

# list_class = [df_class_noPCA, df_class_PCA60, df_class_PCA70, df_class_PCA80, df_class_PCA90, df_class_PCA95, df_class_PCA99]

# list_perf = ['alg_set-2_single_noPCA.csv',
#             'alg_set-2_single_60PCA.csv',
#             'alg_set-2_single_70PCA.csv',
#             'alg_set-2_single_80PCA.csv',
#             'alg_set-2_single_90PCA.csv',
#             'alg_set-2_single_95PCA.csv',
#             'alg_set-2_single_99PCA.csv',]

# list_class = [ 'alg_set-2_class_single_noPCA.csv',
#            'alg_set-2_class_single_60PCA.csv',
#            'alg_set-2_class_single_70PCA.csv',
#            'alg_set-2_class_single_80PCA.csv',
#            'alg_set-2_class_single_90PCA.csv',
#            'alg_set-2_class_single_95PCA.csv',
#            'alg_set-2_class_single_99PCA.csv',]

# list_class = [ 'alg_set-1_class_single_noPCA.csv',
#            'alg_set-1_class_single_60PCA.csv',
#            'alg_set-1_class_single_70PCA.csv',
#            'alg_set-1_class_single_80PCA.csv',
#            'alg_set-1_class_single_90PCA.csv',
#            'alg_set-1_class_single_95PCA.csv',
#            'alg_set-1_class_single_99PCA.csv',]

# for i, out_df in enumerate(list_class):

#     fn = test_dir / list_perf[i]
#     fn = test_dir / list_class[i]
#     out_df.to_csv(fn)
#     print('Saved to {}'.format(fn))


In [ ]:
## prepare training & validation set - single grid 

# subsample 
df_train_1 = df_train_val[ df_train_val[target_col[0]] == 1]
_df_train_0 = df_train_val[ df_train_val[target_col[0]] == 0]
ix_df_train_0 = []

## sample for each 0 sample 
for i, idx in enumerate(_df_train_0['tag'].unique()):
    sub_df_train_0 = _df_train_0[_df_train_0['tag']==idx]
    ## get sample
    subsample = sub_df_train_0.sample(n=1).index.values[0]
    ix_df_train_0.append(subsample)
df_train_0 = df_train_val.loc[ix_df_train_0]
df_train = df_train_1.append(df_train_0)

# add a number of samples not in buffer as well 
df_not_in_buffer = df_train_val[df_train_val['in_buffer']!=1]
n_sample = df_not_in_buffer['tag'].nunique()

df_train = df_train.append( df_not_in_buffer.sample(n=n_sample) ).copy()

print('Division of target values before subsampling: ')
print( df_train_val['target'].value_counts())
print('\nand after subsampling: ')
print(df_train['target'].value_counts())

X_train = df_train[feature_columns]
y_train = df_train[target_col[0]]
y_train_range = df_train['range_target']

# normalize 
min_samples = len(X_train) 
nm = QuantileTransformer( output_distribution='normal',
                        n_quantiles = int(min(1000, min_samples)))
nm.fit(X_train)
X_train = nm.transform(X_train)

# scale 
sc = MinMaxScaler([0,1])
sc.fit(X_train)
X_train = sc.transform(X_train)

# pca 
print(f'\nnumber of features before PCA = {X_train.shape[1]}')
pca = PCA(n_components=0.9)
pca.fit(X_train)
X_train = pca.transform(X_train)
print(f'number of features after PCA = {X_train.shape[1]} with 90% variance')

In [ ]:
## plot PC_i vs PC_j
pc_i = 0
pc_j = 1

ix_0 = np.where(y_train==0)[0]
ix_1 = np.where(y_train==1)[0]

plt.scatter( X_train[ix_0,pc_i], X_train[ix_0,pc_j], color = 'b', label='0' ) 
plt.scatter( X_train[ix_1,pc_i], X_train[ix_1,pc_j], color = 'r', label='1'  ) 

plt.xlabel('PC{}'.format(pc_i+1));
plt.ylabel('PC{}'.format(pc_j+1));
plt.legend();

In [ ]:
## apply same transformations and splits on test set 
X_test = df_test[feature_columns]
y_test = df_test[target_col[0]]
y_test_range = df_test['range_target'] 

## normalize 
X_test = nm.transform(X_test)

## scale 
X_test = sc.transform(X_test)

## pca 
X_test = pca.transform(X_test)

In [ ]:
## train single pixel classifier 

In [ ]:
## MODEL 1 - target in buffer, yes or no?

In [ ]:
## MODEL 2 - if target in buffer, where? 


After finding best settings, use train_val set to train an algorithm, evaluate the final performance with the test set

In [ ]:
##